In [ ]:
import pandas as pd
import numpy as np
from dowhy import CausalModel
import lingam
from lingam.utils import make_dot, remove_effect, make_prior_knowledge
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.2e}'.format(x) if abs(x) < 0.01 else '{:.6f}'.format(x))

def extract_edges(adj_matrix, min_edge_strength=0.0):
    edges = []
    
    for outcome_var in adj_matrix.index:
        for treatment_var in adj_matrix.columns:
            edge_weight = adj_matrix.loc[outcome_var, treatment_var]
            
            if abs(edge_weight) > min_edge_strength and not np.isnan(edge_weight):
                edges.append({
                    'treatment': treatment_var,
                    'outcome': outcome_var,
                    'lingam_weight': edge_weight
                })
    
    return pd.DataFrame(edges)

def estimate_edge_effect(data, treatment_var, outcome_var):
    try:
        clean_data = data[[treatment_var, outcome_var]].dropna()
        
        if clean_data[treatment_var].nunique() < 2:
            return None
        
        graph = f"""
        digraph {{
            {treatment_var} -> {outcome_var};
        }}
        """
        
        model = CausalModel(
            data=clean_data,
            treatment=treatment_var,
            outcome=outcome_var,
            graph=graph
        )
        
        identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
        
        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            method_params={"need_result": True}
        )
        
        estimator = getattr(estimate, "estimator", None)
        
        if estimator is not None and hasattr(estimator, "model"):
            pvalue = float(estimator.model.pvalues[1])
            ci_lower = float(estimator.model.conf_int().iloc[1, 0])
            ci_upper = float(estimator.model.conf_int().iloc[1, 1])
        else:
            pvalue = np.nan
            ci_lower = np.nan
            ci_upper = np.nan
        
        result = {
            'treatment': treatment_var,
            'outcome': outcome_var,
            'sample_size': len(clean_data),
            'effect': estimate.value,
            'pvalue': pvalue,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        }
        
        return result
        
    except Exception as e:
        print(f"Error analyzing {treatment_var} -> {outcome_var}: {str(e)}")
        return None

def run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges):
    all_variables = exog_vars + endog_vars + [sink_var]
    data_exp = data[all_variables].copy()
    data_clean = data_exp.dropna(axis=1)
    
    remaining_vars = data_clean.columns.tolist()
    exog_var_names = [v for v in exog_vars if v in remaining_vars]
    endog_var_names = [v for v in endog_vars if v in remaining_vars]
    
    X = data_clean.values
    exog_indices = [data_clean.columns.tolist().index(v) for v in exog_var_names]
    endog_indices = [data_clean.columns.tolist().index(v) for v in endog_var_names]
    sink_index = data_clean.columns.tolist().index(sink_var)
    
    X_removed_exog, W_endog_dict = remove_effect(X, exog_indices, return_coefs=True)
    del W_endog_dict[sink_index]
    
    W_endog = pd.DataFrame([W_endog_dict[i] for i in endog_indices],
        index=endog_var_names, columns=exog_var_names)
    
    for edge in forbidden_edges:
        if edge['treatment'] in exog_var_names and edge['outcome'] in endog_var_names:
            W_endog.loc[edge['outcome'], edge['treatment']] = 0.0
    
    X_endog = X_removed_exog[:, endog_indices]
    
    no_paths = []
    for edge in forbidden_edges:
        if edge['treatment'] in endog_var_names and edge['outcome'] in endog_var_names:
            from_idx = endog_var_names.index(edge['treatment'])
            to_idx = endog_var_names.index(edge['outcome'])
            no_paths.append([from_idx, to_idx])
    
    if no_paths:
        prior_knowledge = make_prior_knowledge(
            n_variables=len(endog_var_names),
            no_paths=no_paths
        )
        model = lingam.DirectLiNGAM(prior_knowledge=prior_knowledge)
    else:
        model = lingam.DirectLiNGAM()
    
    model.fit(X_endog)
    
    B_endog = pd.DataFrame(model.adjacency_matrix_, 
        index=endog_var_names, columns=endog_var_names)
    
    sink_model = LogisticRegression(max_iter=10000)
    sink_model.fit(X_removed_exog[:, endog_indices], X[:, sink_index])
    
    W_sink = pd.DataFrame(sink_model.coef_, 
        index=[sink_var], columns=endog_var_names)
    
    B_exog = pd.DataFrame(np.eye(len(exog_var_names)) - 1,
        index=exog_var_names, columns=exog_var_names)
    B_exog[B_exog == -1] = np.nan
    
    all_var_names = exog_var_names + endog_var_names + [sink_var]
    adj = pd.DataFrame(0.0, index=all_var_names, columns=all_var_names)
    adj.loc[B_exog.index, B_exog.columns] = B_exog
    adj.loc[W_endog.index, W_endog.columns] = W_endog
    adj.loc[B_endog.index, B_endog.columns] = B_endog
    adj.loc[W_sink.index, W_sink.columns] = W_sink
    
    return adj

def make_labeled_graph(matrix, labels, value_label):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    color = 'red' if value_label == 'Weight' else 'blue'
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            val = matrix.iloc[i, j]
            
            if abs(val) > 0 and not np.isnan(val):
                edge_label = f'<<FONT COLOR="{color}">{val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=edge_label)
    
    return d

def make_dual_value_graph(adj_matrix, effect_matrix, labels):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            lingam_val = adj_matrix.iloc[i, j]
            effect_val = effect_matrix.iloc[i, j]
            
            if abs(lingam_val) > 0 and not np.isnan(lingam_val):
                label_text = f'<<FONT COLOR="blue"> {lingam_val:.2f}</FONT><BR/><FONT COLOR="red">{effect_val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=label_text)
    
    return d

def analyze_edges_from_matrix(adj_matrix, data):
    edges_df = extract_edges(adj_matrix, min_edge_strength=0.0)
    
    results = []
    
    for idx, edge in edges_df.iterrows():
        treatment = edge['treatment']
        outcome = edge['outcome']
        lingam_weight = edge['lingam_weight']
        
        result = estimate_edge_effect(data, treatment, outcome)
        
        if result:
            result['lingam_weight'] = lingam_weight
            results.append(result)
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    results_df['significant'] = results_df['pvalue'] < 0.05
    
    return results_df

def iterative_pruning_with_lingam(data, exog_vars, endog_vars, sink_var, experiment_name, significance_level=0.05):
    print(f"{'='*80}")
    print(f"ITERATIVE PRUNING WITH LINGAM RERUN: {experiment_name}")
    print(f"{'='*80}")
    
    forbidden_edges = []
    iteration = 0
    all_iterations = []
    all_adj_matrices = []
    all_effect_matrices = []
    
    while True:
        iteration += 1
        print(f"{'='*80}")
        print(f"ITERATION {iteration}")
        print(f"{'='*80}")
        print(f"Forbidden edges so far: {len(forbidden_edges)}")
        
        print("Running LiNGAM with constraints")
        adj_matrix = run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges)
        
        all_adj_matrices.append((iteration, adj_matrix.copy()))
        
        print("Running DoWhy on all edges")
        results_df = analyze_edges_from_matrix(adj_matrix, data)
        
        if results_df is None or len(results_df) == 0:
            print("No edges remaining to analyze")
            break
        
        effect_matrix = adj_matrix.copy()
        for idx, row in results_df.iterrows():
            effect_matrix.loc[row['outcome'], row['treatment']] = row['effect']
        
        all_effect_matrices.append((iteration, effect_matrix.copy()))
        
        num_edges = len(results_df)
        num_significant = results_df['significant'].sum()
        num_nonsignificant = num_edges - num_significant
        
        print(f"Total edges: {num_edges}")
        print(f"Significant edges (p < {significance_level}): {num_significant}")
        print(f"Non-significant edges: {num_nonsignificant}")
        
        results_df['iteration'] = iteration
        all_iterations.append(results_df)
        
        nonsig_edges = results_df[~results_df['significant']]
        
        nonsig_edges_maskable = nonsig_edges[
            ((nonsig_edges['treatment'].isin(endog_vars)) & 
             ((nonsig_edges['outcome'].isin(endog_vars)) | (nonsig_edges['outcome'] == sink_var))) |
            ((nonsig_edges['treatment'].isin(exog_vars)) & (nonsig_edges['outcome'].isin(endog_vars)))
        ]
        
        if len(nonsig_edges_maskable) == 0:
            print(f"All edges are significant! Pruning complete.")
            break
        
        worst_edge = nonsig_edges_maskable.loc[nonsig_edges_maskable['pvalue'].idxmax()]
        
        print(f"Removing edge with highest p-value:")
        print(f"  {worst_edge['treatment']} -> {worst_edge['outcome']}")
        print(f"  p-value: {worst_edge['pvalue']:.6f}")
        print(f"  effect: {worst_edge['effect']:.6f}")
        
        new_edge = {
            'treatment': worst_edge['treatment'],
            'outcome': worst_edge['outcome']
        }
        
        if new_edge in forbidden_edges:
            print(f"WARNING: Infinite loop detected! Edge {new_edge['treatment']} -> {new_edge['outcome']} already forbidden.")
            print(f"This edge cannot be removed from the graph. Stopping pruning.")
            break
            
        forbidden_edges.append(new_edge)
    
    print(f"{'='*80}")
    print(f"PRUNING COMPLETE AFTER {iteration} ITERATIONS")
    print(f"{'='*80}")
    
    final_results = all_iterations[-1] if all_iterations else None
    
    if final_results is not None:
        print(f"Final network has {len(final_results)} edges")
        print(f"All edges are significant (p < {significance_level})")
        
        combined_results = pd.concat(all_iterations, ignore_index=True)
        combined_results.to_csv(f'{experiment_name}_results.csv', index=False)
        print(f"Saved: {experiment_name}_results.csv")
        
        all_adj_dfs = []
        for iter_num, adj_mat in all_adj_matrices:
            adj_stacked = adj_mat.stack().reset_index()
            adj_stacked.columns = ['outcome', 'treatment', 'weight']
            adj_stacked['iteration'] = iter_num
            all_adj_dfs.append(adj_stacked)
        
        combined_adjacency = pd.concat(all_adj_dfs, ignore_index=True)
        combined_adjacency.to_csv(f'{experiment_name}_adjacency.csv', index=False)
        print(f"Saved: {experiment_name}_adjacency.csv")
        
        print("Generating PDFs")
        
        try:
            from PyPDF2 import PdfMerger
            import os
            
            merger_lingam = PdfMerger()
            merger_dowhy = PdfMerger()
            temp_files = []
            
            for iter_num, adj_mat in all_adj_matrices:
                dot = make_labeled_graph(adj_mat, adj_mat.columns.tolist(), 'Weight')
                temp_file = f'{experiment_name}_temp_lingam_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_lingam.append(f'{temp_file}.pdf')
            
            for iter_num, eff_mat in all_effect_matrices:
                dot = make_labeled_graph(eff_mat, eff_mat.columns.tolist(), 'Effect')
                temp_file = f'{experiment_name}_temp_dowhy_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_dowhy.append(f'{temp_file}.pdf')
            
            output_pdf_lingam = f'{experiment_name}_iterations_lingam.pdf'
            merger_lingam.write(output_pdf_lingam)
            merger_lingam.close()
            print(f"Saved: {output_pdf_lingam}")
            
            output_pdf_dowhy = f'{experiment_name}_iterations_dowhy.pdf'
            merger_dowhy.write(output_pdf_dowhy)
            merger_dowhy.close()
            print(f"Saved: {output_pdf_dowhy}")
            
            for temp_file in temp_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            

            # Create first iteration graphs (initial state before pruning)
            print("Creating first iteration graphs")
            first_adj_lingam = all_adj_matrices[0][1].copy()
            first_adj_dowhy = all_effect_matrices[0][1].copy()
            
            dot_first_lingam = make_labeled_graph(first_adj_lingam, first_adj_lingam.columns.tolist(), 'Weight')
            dot_first_lingam.render(f'{experiment_name}_first_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_lingam.pdf")
            
            dot_first_dowhy = make_labeled_graph(first_adj_dowhy, first_adj_dowhy.columns.tolist(), 'Effect')
            dot_first_dowhy.render(f'{experiment_name}_first_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dowhy.pdf")
            
            dot_dual_first = make_dual_value_graph(first_adj_lingam, first_adj_dowhy, first_adj_lingam.columns.tolist())
            dot_dual_first.render(f'{experiment_name}_first_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dual.pdf")
            
            # Save first iteration results to CSV
            if len(all_iterations) > 0:
                first_results = all_iterations[0].copy()
                first_results.to_csv(f'{experiment_name}_first_results.csv', index=False)
                print(f"Saved: {experiment_name}_first_results.csv")
            
            print("Creating final graphs")
            significant_edges = final_results[final_results['significant']]
            
            sig_adj_lingam = all_adj_matrices[-1][1].copy()
            sig_adj_dowhy = all_effect_matrices[-1][1].copy()
            
            for idx, row in final_results[~final_results['significant']].iterrows():
                sig_adj_lingam.loc[row['outcome'], row['treatment']] = 0.0
                sig_adj_dowhy.loc[row['outcome'], row['treatment']] = 0.0
            
            dot_final_lingam = make_labeled_graph(sig_adj_lingam, sig_adj_lingam.columns.tolist(), 'Weight')
            dot_final_lingam.render(f'{experiment_name}_final_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_lingam.pdf")
            
            dot_final_dowhy = make_labeled_graph(sig_adj_dowhy, sig_adj_dowhy.columns.tolist(), 'Effect')
            dot_final_dowhy.render(f'{experiment_name}_final_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dowhy.pdf")
            
            dot_dual_final = make_dual_value_graph(sig_adj_lingam, sig_adj_dowhy, sig_adj_lingam.columns.tolist())
            dot_dual_final.render(f'{experiment_name}_final_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dual.pdf")
            
            print(f"Final network has {len(significant_edges)} significant edges")
            
        except ImportError:
            print("PyPDF2 not installed. Install with 'pip install PyPDF2' to create combined PDF.")
    
    return adj_matrix, final_results, forbidden_edges

data_1 = pd.read_csv('Figure_2_variables.csv')
data_2 = pd.read_csv('Figure_3_variables.csv')

all_data = pd.concat([data_1, data_2], axis=1)
all_data = all_data.loc[:, ~all_data.columns.duplicated(keep='first')]
all_data = all_data[all_data['alcohol_category'].notna()]

organ_injury_mediators = ['cardio28', 'cns28', 'coag28', 'hepatic28', 'icufd',
                          'orgfree28', 'pulmon28', 'renal28', 'vfd', 'cirrc']

additional_mediators = ['album_12', 'bili7', 'cumfl_8', 'fio2abg_7', 'flbal_7',
    'fluidin_8', 'gcs_7', 'gluc_2', 'goalrate_0', 'hgb_7', 'mg_2', 'minvent_4', 
    'motorc_14', 'oi_3', 'peep_1', 'pf7', 'phabg_4', 'phos_1', 'pip_7', 'potas_2', 'tidal_12']

all_mediators = organ_injury_mediators + additional_mediators
sink_var_name = 'death90'

print("" + "="*80)
print("RUNNING FOR ALPHA = 0.05")
print("="*80)

# CATEGORICAL GROUPS COMMENTED OUT - Using continuous drinks_per_week only
# data_group1 = all_data.copy()
# data_group1['alcohol_binary'] = data_group1['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Heavy Drinker': 1
# })
# data_group1 = data_group1.dropna(subset=['alcohol_binary'])
# data_group1['alcohol_binary'] = data_group1['alcohol_binary'].astype(int)

# final_adj_group1, results_group1, forbidden_group1 = iterative_pruning_with_lingam(
#     data=data_group1,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='group1_nondrinker_vs_heavy',
#     significance_level=0.05
# )

# data_group3 = all_data.copy()
# data_group3['alcohol_binary'] = data_group3['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1,
#     'Heavy Drinker': 1
# })
# data_group3 = data_group3.dropna(subset=['alcohol_binary'])
# data_group3['alcohol_binary'] = data_group3['alcohol_binary'].astype(int)

# final_adj_group3, results_group3, forbidden_group3 = iterative_pruning_with_lingam(
#     data=data_group3,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='group3_nondrinker_vs_any',
#     significance_level=0.05
# )

# data_group4 = all_data.copy()
# data_group4['alcohol_binary'] = data_group4['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1
# })
# data_group4 = data_group4.dropna(subset=['alcohol_binary'])
# data_group4['alcohol_binary'] = data_group4['alcohol_binary'].astype(int)

# final_adj_group4, results_group4, forbidden_group4 = iterative_pruning_with_lingam(
#     data=data_group4,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='group4_nondrinker_vs_moderate',
#     significance_level=0.05
# )

print("" + "="*80)
print("ALL PRUNING COMPLETE")
print("="*80)

/home/arslanbisharat/anaconda3/envs/arslan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



RUNNING FOR ALPHA = 0.05

ITERATIVE PRUNING WITH LINGAM RERUN: group1_nondrinker_vs_heavy

ITERATION 1
Forbidden edges so far: 0
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 44
Significant edges (p < 0.05): 35
Non-significant edges: 9

Removing edge with highest p-value:
  alcohol_binary -> cardio28
  p-value: 0.994455
  effect: -0.005817

ITERATION 2
Forbidden edges so far: 1
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 43
Significant edges (p < 0.05): 35
Non-significant edges: 8

Removing edge with highest p-value:
  alcohol_binary -> hepatic28
  p-value: 0.623969
  effect: -0.413394

ITERATION 3
Forbidden edges so far: 2
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 42
Significant edges (p < 0.05): 35
Non-significant edges: 7

Removing edge with highest p-value:
  alcohol_binary -> cns28
  p-value: 0.568189
  effect: -0.538963

ITERATION 4
Forbidden edges so far: 3
Running LiNGAM with 

In [ ]:
import pandas as pd
import numpy as np
from dowhy import CausalModel
import lingam
from lingam.utils import make_dot, remove_effect, make_prior_knowledge
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.2e}'.format(x) if abs(x) < 0.01 else '{:.6f}'.format(x))

def extract_edges(adj_matrix, min_edge_strength=0.0):
    edges = []
    
    for outcome_var in adj_matrix.index:
        for treatment_var in adj_matrix.columns:
            edge_weight = adj_matrix.loc[outcome_var, treatment_var]
            
            if abs(edge_weight) > min_edge_strength and not np.isnan(edge_weight):
                edges.append({
                    'treatment': treatment_var,
                    'outcome': outcome_var,
                    'lingam_weight': edge_weight
                })
    
    return pd.DataFrame(edges)

def estimate_edge_effect(data, treatment_var, outcome_var):
    try:
        clean_data = data[[treatment_var, outcome_var]].dropna()
        
        if clean_data[treatment_var].nunique() < 2:
            return None
        
        graph = f"""
        digraph {{
            {treatment_var} -> {outcome_var};
        }}
        """
        
        model = CausalModel(
            data=clean_data,
            treatment=treatment_var,
            outcome=outcome_var,
            graph=graph
        )
        
        identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
        
        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            method_params={"need_result": True}
        )
        
        estimator = getattr(estimate, "estimator", None)
        
        if estimator is not None and hasattr(estimator, "model"):
            pvalue = float(estimator.model.pvalues[1])
            ci_lower = float(estimator.model.conf_int().iloc[1, 0])
            ci_upper = float(estimator.model.conf_int().iloc[1, 1])
        else:
            pvalue = np.nan
            ci_lower = np.nan
            ci_upper = np.nan
        
        result = {
            'treatment': treatment_var,
            'outcome': outcome_var,
            'sample_size': len(clean_data),
            'effect': estimate.value,
            'pvalue': pvalue,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        }
        
        return result
        
    except Exception as e:
        print(f"Error analyzing {treatment_var} -> {outcome_var}: {str(e)}")
        return None

def run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges):
    all_variables = exog_vars + endog_vars + [sink_var]
    data_exp = data[all_variables].copy()
    data_clean = data_exp.dropna(axis=1)
    
    remaining_vars = data_clean.columns.tolist()
    exog_var_names = [v for v in exog_vars if v in remaining_vars]
    endog_var_names = [v for v in endog_vars if v in remaining_vars]
    
    X = data_clean.values
    exog_indices = [data_clean.columns.tolist().index(v) for v in exog_var_names]
    endog_indices = [data_clean.columns.tolist().index(v) for v in endog_var_names]
    sink_index = data_clean.columns.tolist().index(sink_var)
    
    X_removed_exog, W_endog_dict = remove_effect(X, exog_indices, return_coefs=True)
    del W_endog_dict[sink_index]
    
    W_endog = pd.DataFrame([W_endog_dict[i] for i in endog_indices],
        index=endog_var_names, columns=exog_var_names)
    
    for edge in forbidden_edges:
        if edge['treatment'] in exog_var_names and edge['outcome'] in endog_var_names:
            W_endog.loc[edge['outcome'], edge['treatment']] = 0.0
    
    X_endog = X_removed_exog[:, endog_indices]
    
    no_paths = []
    for edge in forbidden_edges:
        if edge['treatment'] in endog_var_names and edge['outcome'] in endog_var_names:
            from_idx = endog_var_names.index(edge['treatment'])
            to_idx = endog_var_names.index(edge['outcome'])
            no_paths.append([from_idx, to_idx])
    
    if no_paths:
        prior_knowledge = make_prior_knowledge(
            n_variables=len(endog_var_names),
            no_paths=no_paths
        )
        model = lingam.DirectLiNGAM(prior_knowledge=prior_knowledge)
    else:
        model = lingam.DirectLiNGAM()
    
    model.fit(X_endog)
    
    B_endog = pd.DataFrame(model.adjacency_matrix_, 
        index=endog_var_names, columns=endog_var_names)
    
    sink_model = LogisticRegression(max_iter=10000)
    sink_model.fit(X_removed_exog[:, endog_indices], X[:, sink_index])
    
    W_sink = pd.DataFrame(sink_model.coef_, 
        index=[sink_var], columns=endog_var_names)
    
    B_exog = pd.DataFrame(np.eye(len(exog_var_names)) - 1,
        index=exog_var_names, columns=exog_var_names)
    B_exog[B_exog == -1] = np.nan
    
    all_var_names = exog_var_names + endog_var_names + [sink_var]
    adj = pd.DataFrame(0.0, index=all_var_names, columns=all_var_names)
    adj.loc[B_exog.index, B_exog.columns] = B_exog
    adj.loc[W_endog.index, W_endog.columns] = W_endog
    adj.loc[B_endog.index, B_endog.columns] = B_endog
    adj.loc[W_sink.index, W_sink.columns] = W_sink
    
    return adj

def make_labeled_graph(matrix, labels, value_label):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    color = 'red' if value_label == 'Weight' else 'blue'
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            val = matrix.iloc[i, j]
            
            if abs(val) > 0 and not np.isnan(val):
                edge_label = f'<<FONT COLOR="{color}">{val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=edge_label)
    
    return d

def make_dual_value_graph(adj_matrix, effect_matrix, labels):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            lingam_val = adj_matrix.iloc[i, j]
            effect_val = effect_matrix.iloc[i, j]
            
            if abs(lingam_val) > 0 and not np.isnan(lingam_val):
                label_text = f'<<FONT COLOR="blue"> {lingam_val:.2f}</FONT><BR/><FONT COLOR="red">{effect_val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=label_text)
    
    return d

def analyze_edges_from_matrix(adj_matrix, data):
    edges_df = extract_edges(adj_matrix, min_edge_strength=0.0)
    
    results = []
    
    for idx, edge in edges_df.iterrows():
        treatment = edge['treatment']
        outcome = edge['outcome']
        lingam_weight = edge['lingam_weight']
        
        result = estimate_edge_effect(data, treatment, outcome)
        
        if result:
            result['lingam_weight'] = lingam_weight
            results.append(result)
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    results_df['significant'] = results_df['pvalue'] < 0.05
    
    return results_df

def iterative_pruning_with_lingam(data, exog_vars, endog_vars, sink_var, experiment_name, significance_level=0.05):
    print(f"{'='*80}")
    print(f"ITERATIVE PRUNING WITH LINGAM RERUN: {experiment_name}")
    print(f"{'='*80}")
    
    forbidden_edges = []
    iteration = 0
    all_iterations = []
    all_adj_matrices = []
    all_effect_matrices = []
    
    while True:
        iteration += 1
        print(f"{'='*80}")
        print(f"ITERATION {iteration}")
        print(f"{'='*80}")
        print(f"Forbidden edges so far: {len(forbidden_edges)}")
        
        print("Running LiNGAM with constraints")
        adj_matrix = run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges)
        
        all_adj_matrices.append((iteration, adj_matrix.copy()))
        
        print("Running DoWhy on all edges")
        results_df = analyze_edges_from_matrix(adj_matrix, data)
        
        if results_df is None or len(results_df) == 0:
            print("No edges remaining to analyze")
            break
        
        effect_matrix = adj_matrix.copy()
        for idx, row in results_df.iterrows():
            effect_matrix.loc[row['outcome'], row['treatment']] = row['effect']
        
        all_effect_matrices.append((iteration, effect_matrix.copy()))
        
        num_edges = len(results_df)
        num_significant = results_df['significant'].sum()
        num_nonsignificant = num_edges - num_significant
        
        print(f"Total edges: {num_edges}")
        print(f"Significant edges (p < {significance_level}): {num_significant}")
        print(f"Non-significant edges: {num_nonsignificant}")
        
        results_df['iteration'] = iteration
        all_iterations.append(results_df)
        
        nonsig_edges = results_df[~results_df['significant']]
        
        nonsig_edges_maskable = nonsig_edges[
            ((nonsig_edges['treatment'].isin(endog_vars)) & 
             ((nonsig_edges['outcome'].isin(endog_vars)) | (nonsig_edges['outcome'] == sink_var))) |
            ((nonsig_edges['treatment'].isin(exog_vars)) & (nonsig_edges['outcome'].isin(endog_vars)))
        ]
        
        if len(nonsig_edges_maskable) == 0:
            print(f"All edges are significant! Pruning complete.")
            break
        
        worst_edge = nonsig_edges_maskable.loc[nonsig_edges_maskable['pvalue'].idxmax()]
        
        print(f"Removing edge with highest p-value:")
        print(f"  {worst_edge['treatment']} -> {worst_edge['outcome']}")
        print(f"  p-value: {worst_edge['pvalue']:.6f}")
        print(f"  effect: {worst_edge['effect']:.6f}")
        
        new_edge = {
            'treatment': worst_edge['treatment'],
            'outcome': worst_edge['outcome']
        }
        
        if new_edge in forbidden_edges:
            print(f"WARNING: Infinite loop detected! Edge {new_edge['treatment']} -> {new_edge['outcome']} already forbidden.")
            print(f"This edge cannot be removed from the graph. Stopping pruning.")
            break
            
        forbidden_edges.append(new_edge)
    
    print(f"{'='*80}")
    print(f"PRUNING COMPLETE AFTER {iteration} ITERATIONS")
    print(f"{'='*80}")
    
    final_results = all_iterations[-1] if all_iterations else None
    
    if final_results is not None:
        print(f"Final network has {len(final_results)} edges")
        print(f"All edges are significant (p < {significance_level})")
        
        combined_results = pd.concat(all_iterations, ignore_index=True)
        combined_results.to_csv(f'{experiment_name}_results.csv', index=False)
        print(f"Saved: {experiment_name}_results.csv")
        
        all_adj_dfs = []
        for iter_num, adj_mat in all_adj_matrices:
            adj_stacked = adj_mat.stack().reset_index()
            adj_stacked.columns = ['outcome', 'treatment', 'weight']
            adj_stacked['iteration'] = iter_num
            all_adj_dfs.append(adj_stacked)
        
        combined_adjacency = pd.concat(all_adj_dfs, ignore_index=True)
        combined_adjacency.to_csv(f'{experiment_name}_adjacency.csv', index=False)
        print(f"Saved: {experiment_name}_adjacency.csv")
        
        print("Generating PDFs")
        
        try:
            from PyPDF2 import PdfMerger
            import os
            
            merger_lingam = PdfMerger()
            merger_dowhy = PdfMerger()
            temp_files = []
            
            for iter_num, adj_mat in all_adj_matrices:
                dot = make_labeled_graph(adj_mat, adj_mat.columns.tolist(), 'Weight')
                temp_file = f'{experiment_name}_temp_lingam_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_lingam.append(f'{temp_file}.pdf')
            
            for iter_num, eff_mat in all_effect_matrices:
                dot = make_labeled_graph(eff_mat, eff_mat.columns.tolist(), 'Effect')
                temp_file = f'{experiment_name}_temp_dowhy_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_dowhy.append(f'{temp_file}.pdf')
            
            output_pdf_lingam = f'{experiment_name}_iterations_lingam.pdf'
            merger_lingam.write(output_pdf_lingam)
            merger_lingam.close()
            print(f"Saved: {output_pdf_lingam}")
            
            output_pdf_dowhy = f'{experiment_name}_iterations_dowhy.pdf'
            merger_dowhy.write(output_pdf_dowhy)
            merger_dowhy.close()
            print(f"Saved: {output_pdf_dowhy}")
            
            for temp_file in temp_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            

            # Create first iteration graphs (initial state before pruning)
            print("Creating first iteration graphs")
            first_adj_lingam = all_adj_matrices[0][1].copy()
            first_adj_dowhy = all_effect_matrices[0][1].copy()
            
            dot_first_lingam = make_labeled_graph(first_adj_lingam, first_adj_lingam.columns.tolist(), 'Weight')
            dot_first_lingam.render(f'{experiment_name}_first_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_lingam.pdf")
            
            dot_first_dowhy = make_labeled_graph(first_adj_dowhy, first_adj_dowhy.columns.tolist(), 'Effect')
            dot_first_dowhy.render(f'{experiment_name}_first_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dowhy.pdf")
            
            dot_dual_first = make_dual_value_graph(first_adj_lingam, first_adj_dowhy, first_adj_lingam.columns.tolist())
            dot_dual_first.render(f'{experiment_name}_first_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dual.pdf")
            
            # Save first iteration results to CSV
            if len(all_iterations) > 0:
                first_results = all_iterations[0].copy()
                first_results.to_csv(f'{experiment_name}_first_results.csv', index=False)
                print(f"Saved: {experiment_name}_first_results.csv")
            
            print("Creating final graphs")
            significant_edges = final_results[final_results['significant']]
            
            sig_adj_lingam = all_adj_matrices[-1][1].copy()
            sig_adj_dowhy = all_effect_matrices[-1][1].copy()
            
            for idx, row in final_results[~final_results['significant']].iterrows():
                sig_adj_lingam.loc[row['outcome'], row['treatment']] = 0.0
                sig_adj_dowhy.loc[row['outcome'], row['treatment']] = 0.0
            
            dot_final_lingam = make_labeled_graph(sig_adj_lingam, sig_adj_lingam.columns.tolist(), 'Weight')
            dot_final_lingam.render(f'{experiment_name}_final_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_lingam.pdf")
            
            dot_final_dowhy = make_labeled_graph(sig_adj_dowhy, sig_adj_dowhy.columns.tolist(), 'Effect')
            dot_final_dowhy.render(f'{experiment_name}_final_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dowhy.pdf")
            
            dot_dual_final = make_dual_value_graph(sig_adj_lingam, sig_adj_dowhy, sig_adj_lingam.columns.tolist())
            dot_dual_final.render(f'{experiment_name}_final_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dual.pdf")
            
            print(f"Final network has {len(significant_edges)} significant edges")
            
        except ImportError:
            print("PyPDF2 not installed. Install with 'pip install PyPDF2' to create combined PDF.")
    
    return adj_matrix, final_results, forbidden_edges

data_1 = pd.read_csv('Figure_2_variables.csv')
data_2 = pd.read_csv('Figure_3_variables.csv')

all_data = pd.concat([data_1, data_2], axis=1)
all_data = all_data.loc[:, ~all_data.columns.duplicated(keep='first')]
all_data = all_data[all_data['drinks_per_week'].notna()]

organ_injury_mediators = ['cardio28', 'cns28', 'coag28', 'hepatic28', 'icufd',
                          'orgfree28', 'pulmon28', 'renal28', 'vfd']

additional_mediators = ['album_12', 'bili7', 'cumfl_8', 'fio2abg_7', 'flbal_7',
    'fluidin_8', 'gcs_7', 'gluc_2', 'goalrate_0', 'hgb_7', 'mg_2', 'minvent_4', 
    'motorc_14', 'oi_3', 'peep_1', 'pf7', 'phabg_4', 'phos_1', 'pip_7', 'potas_2', 'tidal_12']

all_mediators = organ_injury_mediators + additional_mediators
sink_var_name = 'death90'

print("" + "="*80)
print("RUNNING FOR ALPHA = 0.05")
print("="*80)

data_continuous = all_data[['drinks_per_week'] + all_mediators + [sink_var_name]].dropna(subset=['drinks_per_week'])

final_adj_continuous, results_continuous, forbidden_continuous = iterative_pruning_with_lingam(
    data=data_continuous,
    exog_vars=['drinks_per_week'],
    endog_vars=all_mediators,
    sink_var=sink_var_name,
    experiment_name='continuous_drinks_per_week',
    significance_level=0.05
)

print("" + "="*80)
print("ALL PRUNING COMPLETE")
print("="*80)


RUNNING FOR ALPHA = 0.05

ITERATIVE PRUNING WITH LINGAM RERUN: continuous_drinks_per_week

ITERATION 1
Forbidden edges so far: 0
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 44
Significant edges (p < 0.05): 35
Non-significant edges: 9

Removing edge with highest p-value:
  drinks_per_week -> cns28
  p-value: 0.751930
  effect: -0.007528

ITERATION 2
Forbidden edges so far: 1
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 43
Significant edges (p < 0.05): 35
Non-significant edges: 8

Removing edge with highest p-value:
  drinks_per_week -> cardio28
  p-value: 0.610437
  effect: 0.010387

ITERATION 3
Forbidden edges so far: 2
Running LiNGAM with constraints...
Running DoWhy on all edges...
Total edges: 42
Significant edges (p < 0.05): 35
Non-significant edges: 7

Removing edge with highest p-value:
  drinks_per_week -> vfd
  p-value: 0.483539
  effect: -0.015790

ITERATION 4
Forbidden edges so far: 3
Running LiNGAM with cons

In [ ]:
import pandas as pd
import numpy as np
from dowhy import CausalModel
import lingam
from lingam.utils import make_dot, remove_effect, make_prior_knowledge
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.2e}'.format(x) if abs(x) < 0.01 else '{:.6f}'.format(x))

def extract_edges(adj_matrix, min_edge_strength=0.0):
    edges = []
    
    for outcome_var in adj_matrix.index:
        for treatment_var in adj_matrix.columns:
            edge_weight = adj_matrix.loc[outcome_var, treatment_var]
            
            if abs(edge_weight) > min_edge_strength and not np.isnan(edge_weight):
                edges.append({
                    'treatment': treatment_var,
                    'outcome': outcome_var,
                    'lingam_weight': edge_weight
                })
    
    return pd.DataFrame(edges)

def estimate_edge_effect(data, treatment_var, outcome_var):
    try:
        clean_data = data[[treatment_var, outcome_var]].dropna()
        
        if clean_data[treatment_var].nunique() < 2:
            return None
        
        graph = f"""
        digraph {{
            {treatment_var} -> {outcome_var};
        }}
        """
        
        model = CausalModel(
            data=clean_data,
            treatment=treatment_var,
            outcome=outcome_var,
            graph=graph
        )
        
        identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
        
        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            method_params={"need_result": True}
        )
        
        estimator = getattr(estimate, "estimator", None)
        
        if estimator is not None and hasattr(estimator, "model"):
            pvalue = float(estimator.model.pvalues[1])
            ci_lower = float(estimator.model.conf_int().iloc[1, 0])
            ci_upper = float(estimator.model.conf_int().iloc[1, 1])
        else:
            pvalue = np.nan
            ci_lower = np.nan
            ci_upper = np.nan
        
        result = {
            'treatment': treatment_var,
            'outcome': outcome_var,
            'sample_size': len(clean_data),
            'effect': estimate.value,
            'pvalue': pvalue,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        }
        
        return result
        
    except Exception as e:
        print(f"Error analyzing {treatment_var} -> {outcome_var}: {str(e)}")
        return None

def run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges):
    all_variables = exog_vars + endog_vars + [sink_var]
    data_exp = data[all_variables].copy()
    data_clean = data_exp.dropna(axis=1)
    
    remaining_vars = data_clean.columns.tolist()
    exog_var_names = [v for v in exog_vars if v in remaining_vars]
    endog_var_names = [v for v in endog_vars if v in remaining_vars]
    
    X = data_clean.values
    exog_indices = [data_clean.columns.tolist().index(v) for v in exog_var_names]
    endog_indices = [data_clean.columns.tolist().index(v) for v in endog_var_names]
    sink_index = data_clean.columns.tolist().index(sink_var)
    
    X_removed_exog, W_endog_dict = remove_effect(X, exog_indices, return_coefs=True)
    del W_endog_dict[sink_index]
    
    W_endog = pd.DataFrame([W_endog_dict[i] for i in endog_indices],
        index=endog_var_names, columns=exog_var_names)
    
    for edge in forbidden_edges:
        if edge['treatment'] in exog_var_names and edge['outcome'] in endog_var_names:
            W_endog.loc[edge['outcome'], edge['treatment']] = 0.0
    
    X_endog = X_removed_exog[:, endog_indices]
    
    no_paths = []
    for edge in forbidden_edges:
        if edge['treatment'] in endog_var_names and edge['outcome'] in endog_var_names:
            from_idx = endog_var_names.index(edge['treatment'])
            to_idx = endog_var_names.index(edge['outcome'])
            no_paths.append([from_idx, to_idx])
    
    if no_paths:
        prior_knowledge = make_prior_knowledge(
            n_variables=len(endog_var_names),
            no_paths=no_paths
        )
        model = lingam.DirectLiNGAM(prior_knowledge=prior_knowledge)
    else:
        model = lingam.DirectLiNGAM()
    
    model.fit(X_endog)
    
    B_endog = pd.DataFrame(model.adjacency_matrix_, 
        index=endog_var_names, columns=endog_var_names)
    
    sink_model = LogisticRegression(max_iter=10000)
    sink_model.fit(X_removed_exog[:, endog_indices], X[:, sink_index])
    
    W_sink = pd.DataFrame(sink_model.coef_, 
        index=[sink_var], columns=endog_var_names)
    
    B_exog = pd.DataFrame(np.eye(len(exog_var_names)) - 1,
        index=exog_var_names, columns=exog_var_names)
    B_exog[B_exog == -1] = np.nan
    
    all_var_names = exog_var_names + endog_var_names + [sink_var]
    adj = pd.DataFrame(0.0, index=all_var_names, columns=all_var_names)
    adj.loc[B_exog.index, B_exog.columns] = B_exog
    adj.loc[W_endog.index, W_endog.columns] = W_endog
    adj.loc[B_endog.index, B_endog.columns] = B_endog
    adj.loc[W_sink.index, W_sink.columns] = W_sink
    
    return adj

def make_labeled_graph(matrix, labels, value_label):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    color = 'red' if value_label == 'Weight' else 'blue'
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            val = matrix.iloc[i, j]
            
            if abs(val) > 0 and not np.isnan(val):
                edge_label = f'<<FONT COLOR="{color}">{val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=edge_label)
    
    return d

def make_dual_value_graph(adj_matrix, effect_matrix, labels):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            lingam_val = adj_matrix.iloc[i, j]
            effect_val = effect_matrix.iloc[i, j]
            
            if abs(lingam_val) > 0 and not np.isnan(lingam_val):
                label_text = f'<<FONT COLOR="blue"> {lingam_val:.2f}</FONT><BR/><FONT COLOR="red">{effect_val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=label_text)
    
    return d

def analyze_edges_from_matrix(adj_matrix, data):
    edges_df = extract_edges(adj_matrix, min_edge_strength=0.0)
    
    results = []
    
    for idx, edge in edges_df.iterrows():
        treatment = edge['treatment']
        outcome = edge['outcome']
        lingam_weight = edge['lingam_weight']
        
        result = estimate_edge_effect(data, treatment, outcome)
        
        if result:
            result['lingam_weight'] = lingam_weight
            results.append(result)
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    results_df['significant'] = results_df['pvalue'] < 0.05
    
    return results_df

def iterative_pruning_with_lingam(data, exog_vars, endog_vars, sink_var, experiment_name, significance_level=0.05):
    print(f"{'='*80}")
    print(f"ITERATIVE PRUNING WITH LINGAM RERUN: {experiment_name}")
    print(f"{'='*80}")
    
    forbidden_edges = []
    iteration = 0
    all_iterations = []
    all_adj_matrices = []
    all_effect_matrices = []
    
    while True:
        iteration += 1
        print(f"{'='*80}")
        print(f"ITERATION {iteration}")
        print(f"{'='*80}")
        print(f"Forbidden edges so far: {len(forbidden_edges)}")
        
        print("Running LiNGAM with constraints")
        adj_matrix = run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges)
        
        all_adj_matrices.append((iteration, adj_matrix.copy()))
        
        print("Running DoWhy on all edges")
        results_df = analyze_edges_from_matrix(adj_matrix, data)
        
        if results_df is None or len(results_df) == 0:
            print("No edges remaining to analyze")
            break
        
        effect_matrix = adj_matrix.copy()
        for idx, row in results_df.iterrows():
            effect_matrix.loc[row['outcome'], row['treatment']] = row['effect']
        
        all_effect_matrices.append((iteration, effect_matrix.copy()))
        
        num_edges = len(results_df)
        num_significant = results_df['significant'].sum()
        num_nonsignificant = num_edges - num_significant
        
        print(f"Total edges: {num_edges}")
        print(f"Significant edges (p < {significance_level}): {num_significant}")
        print(f"Non-significant edges: {num_nonsignificant}")
        
        results_df['iteration'] = iteration
        all_iterations.append(results_df)
        
        nonsig_edges = results_df[~results_df['significant']]
        
        nonsig_edges_maskable = nonsig_edges[
            ((nonsig_edges['treatment'].isin(endog_vars)) & 
             ((nonsig_edges['outcome'].isin(endog_vars)) | (nonsig_edges['outcome'] == sink_var))) |
            ((nonsig_edges['treatment'].isin(exog_vars)) & (nonsig_edges['outcome'].isin(endog_vars)))
        ]
        
        if len(nonsig_edges_maskable) == 0:
            print(f"All edges are significant! Pruning complete.")
            break
        
        worst_edge = nonsig_edges_maskable.loc[nonsig_edges_maskable['pvalue'].idxmax()]
        
        print(f"Removing edge with highest p-value:")
        print(f"  {worst_edge['treatment']} -> {worst_edge['outcome']}")
        print(f"  p-value: {worst_edge['pvalue']:.6f}")
        print(f"  effect: {worst_edge['effect']:.6f}")
        
        new_edge = {
            'treatment': worst_edge['treatment'],
            'outcome': worst_edge['outcome']
        }
        
        if new_edge in forbidden_edges:
            print(f"WARNING: Infinite loop detected! Edge {new_edge['treatment']} -> {new_edge['outcome']} already forbidden.")
            print(f"This edge cannot be removed from the graph. Stopping pruning.")
            break
            
        forbidden_edges.append(new_edge)
    
    print(f"{'='*80}")
    print(f"PRUNING COMPLETE AFTER {iteration} ITERATIONS")
    print(f"{'='*80}")
    
    final_results = all_iterations[-1] if all_iterations else None
    
    if final_results is not None:
        print(f"Final network has {len(final_results)} edges")
        print(f"All edges are significant (p < {significance_level})")
        
        combined_results = pd.concat(all_iterations, ignore_index=True)
        combined_results.to_csv(f'{experiment_name}_results.csv', index=False)
        print(f"Saved: {experiment_name}_results.csv")
        
        all_adj_dfs = []
        for iter_num, adj_mat in all_adj_matrices:
            adj_stacked = adj_mat.stack().reset_index()
            adj_stacked.columns = ['outcome', 'treatment', 'weight']
            adj_stacked['iteration'] = iter_num
            all_adj_dfs.append(adj_stacked)
        
        combined_adjacency = pd.concat(all_adj_dfs, ignore_index=True)
        combined_adjacency.to_csv(f'{experiment_name}_adjacency.csv', index=False)
        print(f"Saved: {experiment_name}_adjacency.csv")
        
        print("Generating PDFs")
        
        try:
            from PyPDF2 import PdfMerger
            import os
            
            merger_lingam = PdfMerger()
            merger_dowhy = PdfMerger()
            temp_files = []
            
            for iter_num, adj_mat in all_adj_matrices:
                dot = make_labeled_graph(adj_mat, adj_mat.columns.tolist(), 'Weight')
                temp_file = f'{experiment_name}_temp_lingam_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_lingam.append(f'{temp_file}.pdf')
            
            for iter_num, eff_mat in all_effect_matrices:
                dot = make_labeled_graph(eff_mat, eff_mat.columns.tolist(), 'Effect')
                temp_file = f'{experiment_name}_temp_dowhy_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_dowhy.append(f'{temp_file}.pdf')
            
            output_pdf_lingam = f'{experiment_name}_iterations_lingam.pdf'
            merger_lingam.write(output_pdf_lingam)
            merger_lingam.close()
            print(f"Saved: {output_pdf_lingam}")
            
            output_pdf_dowhy = f'{experiment_name}_iterations_dowhy.pdf'
            merger_dowhy.write(output_pdf_dowhy)
            merger_dowhy.close()
            print(f"Saved: {output_pdf_dowhy}")
            
            for temp_file in temp_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            

            # Create first iteration graphs (initial state before pruning)
            print("Creating first iteration graphs")
            first_adj_lingam = all_adj_matrices[0][1].copy()
            first_adj_dowhy = all_effect_matrices[0][1].copy()
            
            dot_first_lingam = make_labeled_graph(first_adj_lingam, first_adj_lingam.columns.tolist(), 'Weight')
            dot_first_lingam.render(f'{experiment_name}_first_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_lingam.pdf")
            
            dot_first_dowhy = make_labeled_graph(first_adj_dowhy, first_adj_dowhy.columns.tolist(), 'Effect')
            dot_first_dowhy.render(f'{experiment_name}_first_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dowhy.pdf")
            
            dot_dual_first = make_dual_value_graph(first_adj_lingam, first_adj_dowhy, first_adj_lingam.columns.tolist())
            dot_dual_first.render(f'{experiment_name}_first_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dual.pdf")
            
            # Save first iteration results to CSV
            if len(all_iterations) > 0:
                first_results = all_iterations[0].copy()
                first_results.to_csv(f'{experiment_name}_first_results.csv', index=False)
                print(f"Saved: {experiment_name}_first_results.csv")
            
            print("Creating final graphs")
            significant_edges = final_results[final_results['significant']]
            
            sig_adj_lingam = all_adj_matrices[-1][1].copy()
            sig_adj_dowhy = all_effect_matrices[-1][1].copy()
            
            for idx, row in final_results[~final_results['significant']].iterrows():
                sig_adj_lingam.loc[row['outcome'], row['treatment']] = 0.0
                sig_adj_dowhy.loc[row['outcome'], row['treatment']] = 0.0
            
            dot_final_lingam = make_labeled_graph(sig_adj_lingam, sig_adj_lingam.columns.tolist(), 'Weight')
            dot_final_lingam.render(f'{experiment_name}_final_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_lingam.pdf")
            
            dot_final_dowhy = make_labeled_graph(sig_adj_dowhy, sig_adj_dowhy.columns.tolist(), 'Effect')
            dot_final_dowhy.render(f'{experiment_name}_final_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dowhy.pdf")
            
            dot_dual_final = make_dual_value_graph(sig_adj_lingam, sig_adj_dowhy, sig_adj_lingam.columns.tolist())
            dot_dual_final.render(f'{experiment_name}_final_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dual.pdf")
            
            print(f"Final network has {len(significant_edges)} significant edges")
            
        except ImportError:
            print("PyPDF2 not installed. Install with 'pip install PyPDF2' to create combined PDF.")
    
    return adj_matrix, final_results, forbidden_edges

data_male = pd.read_csv('data_male.csv')
data_female = pd.read_csv('data_female.csv')

organ_injury_mediators = ['cardio28', 'cns28', 'coag28', 'hepatic28', 'icufd',
                          'orgfree28', 'pulmon28', 'renal28', 'vfd']

additional_mediators = ['album_12', 'bili7', 'cumfl_8', 'fio2abg_7', 'flbal_7',
    'fluidin_8', 'gcs_7', 'gluc_2', 'goalrate_0', 'hgb_7', 'mg_2', 'minvent_4', 
    'motorc_14', 'oi_3', 'peep_1', 'pf7', 'phabg_4', 'phos_1', 'pip_7', 'potas_2', 'tidal_12']

all_mediators = organ_injury_mediators + additional_mediators
sink_var_name = 'death90'

print("" + "="*80)
print("RUNNING GENDER-STRATIFIED ANALYSIS FOR ALPHA = 0.05")
print("="*80)

print("" + "="*80)
print("MALE EXPERIMENTS")
print("="*80)

data_male_continuous = data_male[data_male['drinks_per_week'].notna()].copy()
data_male_continuous = data_male_continuous[['drinks_per_week'] + all_mediators + [sink_var_name]].dropna(subset=['drinks_per_week'])

final_adj_male_cont, results_male_cont, forbidden_male_cont = iterative_pruning_with_lingam(
    data=data_male_continuous,
    exog_vars=['drinks_per_week'],
    endog_vars=all_mediators,
    sink_var=sink_var_name,
    experiment_name='gender_male_continuous_drinks_per_week',
    significance_level=0.01
)

# CATEGORICAL GROUPS COMMENTED OUT - Using continuous drinks_per_week only
# data_male_cat = data_male[data_male['alcohol_category'].notna()].copy()

# data_male_group1 = data_male_cat.copy()
# data_male_group1['alcohol_binary'] = data_male_group1['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Heavy Drinker': 1
# })
# data_male_group1 = data_male_group1.dropna(subset=['alcohol_binary'])
# data_male_group1['alcohol_binary'] = data_male_group1['alcohol_binary'].astype(int)

# final_adj_male_g1, results_male_g1, forbidden_male_g1 = iterative_pruning_with_lingam(
#     data=data_male_group1,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_male_group1_nondrinker_vs_heavy',
#     significance_level=0.01
# )

# data_male_group3 = data_male_cat.copy()
# data_male_group3['alcohol_binary'] = data_male_group3['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1,
#     'Heavy Drinker': 1
# })
# data_male_group3 = data_male_group3.dropna(subset=['alcohol_binary'])
# data_male_group3['alcohol_binary'] = data_male_group3['alcohol_binary'].astype(int)

# final_adj_male_g3, results_male_g3, forbidden_male_g3 = iterative_pruning_with_lingam(
#     data=data_male_group3,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_male_group3_nondrinker_vs_anydrinker',
#     significance_level=0.01
# )

# data_male_group4 = data_male_cat.copy()
# data_male_group4['alcohol_binary'] = data_male_group4['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1
# })
# data_male_group4 = data_male_group4.dropna(subset=['alcohol_binary'])
# data_male_group4['alcohol_binary'] = data_male_group4['alcohol_binary'].astype(int)

# final_adj_male_g4, results_male_g4, forbidden_male_g4 = iterative_pruning_with_lingam(
#     data=data_male_group4,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_male_group4_nondrinker_vs_moderate',
#     significance_level=0.01
# )

print("" + "="*80)
print("FEMALE EXPERIMENTS")
print("="*80)

data_female_continuous = data_female[data_female['drinks_per_week'].notna()].copy()
data_female_continuous = data_female_continuous[['drinks_per_week'] + all_mediators + [sink_var_name]].dropna(subset=['drinks_per_week'])

final_adj_female_cont, results_female_cont, forbidden_female_cont = iterative_pruning_with_lingam(
    data=data_female_continuous,
    exog_vars=['drinks_per_week'],
    endog_vars=all_mediators,
    sink_var=sink_var_name,
    experiment_name='gender_female_continuous_drinks_per_week',
    significance_level=0.01
)

# CATEGORICAL GROUPS COMMENTED OUT - Using continuous drinks_per_week only
# data_female_cat = data_female[data_female['alcohol_category'].notna()].copy()

# data_female_group1 = data_female_cat.copy()
# data_female_group1['alcohol_binary'] = data_female_group1['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Heavy Drinker': 1
# })
# data_female_group1 = data_female_group1.dropna(subset=['alcohol_binary'])
# data_female_group1['alcohol_binary'] = data_female_group1['alcohol_binary'].astype(int)

# final_adj_female_g1, results_female_g1, forbidden_female_g1 = iterative_pruning_with_lingam(
#     data=data_female_group1,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_female_group1_nondrinker_vs_heavy',
#     significance_level=0.01
# )

# data_female_group3 = data_female_cat.copy()
# data_female_group3['alcohol_binary'] = data_female_group3['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1,
#     'Heavy Drinker': 1
# })
# data_female_group3 = data_female_group3.dropna(subset=['alcohol_binary'])
# data_female_group3['alcohol_binary'] = data_female_group3['alcohol_binary'].astype(int)

# final_adj_female_g3, results_female_g3, forbidden_female_g3 = iterative_pruning_with_lingam(
#     data=data_female_group3,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_female_group3_nondrinker_vs_anydrinker',
#     significance_level=0.01
# )

# data_female_group4 = data_female_cat.copy()
# data_female_group4['alcohol_binary'] = data_female_group4['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1
# })
# data_female_group4 = data_female_group4.dropna(subset=['alcohol_binary'])
# data_female_group4['alcohol_binary'] = data_female_group4['alcohol_binary'].astype(int)

# final_adj_female_g4, results_female_g4, forbidden_female_g4 = iterative_pruning_with_lingam(
#     data=data_female_group4,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='gender_female_group4_nondrinker_vs_moderate',
#     significance_level=0.01
# )

print("" + "="*80)
print("GENDER-STRATIFIED ANALYSIS COMPLETE")
print("="*80)

In [ ]:
import pandas as pd
import numpy as np
from dowhy import CausalModel
import lingam
from lingam.utils import make_dot, remove_effect, make_prior_knowledge
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.2e}'.format(x) if abs(x) < 0.01 else '{:.6f}'.format(x))

def extract_edges(adj_matrix, min_edge_strength=0.0):
    edges = []
    
    for outcome_var in adj_matrix.index:
        for treatment_var in adj_matrix.columns:
            edge_weight = adj_matrix.loc[outcome_var, treatment_var]
            
            if abs(edge_weight) > min_edge_strength and not np.isnan(edge_weight):
                edges.append({
                    'treatment': treatment_var,
                    'outcome': outcome_var,
                    'lingam_weight': edge_weight
                })
    
    return pd.DataFrame(edges)

def estimate_edge_effect(data, treatment_var, outcome_var):
    try:
        clean_data = data[[treatment_var, outcome_var]].dropna()
        
        if clean_data[treatment_var].nunique() < 2:
            return None
        
        graph = f"""
        digraph {{
            {treatment_var} -> {outcome_var};
        }}
        """
        
        model = CausalModel(
            data=clean_data,
            treatment=treatment_var,
            outcome=outcome_var,
            graph=graph
        )
        
        identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
        
        estimate = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.linear_regression",
            method_params={"need_result": True}
        )
        
        estimator = getattr(estimate, "estimator", None)
        
        if estimator is not None and hasattr(estimator, "model"):
            pvalue = float(estimator.model.pvalues[1])
            ci_lower = float(estimator.model.conf_int().iloc[1, 0])
            ci_upper = float(estimator.model.conf_int().iloc[1, 1])
        else:
            pvalue = np.nan
            ci_lower = np.nan
            ci_upper = np.nan
        
        result = {
            'treatment': treatment_var,
            'outcome': outcome_var,
            'sample_size': len(clean_data),
            'effect': estimate.value,
            'pvalue': pvalue,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        }
        
        return result
        
    except Exception as e:
        print(f"Error analyzing {treatment_var} -> {outcome_var}: {str(e)}")
        return None

def run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges):
    all_variables = exog_vars + endog_vars + [sink_var]
    data_exp = data[all_variables].copy()
    data_clean = data_exp.dropna(axis=1)
    
    remaining_vars = data_clean.columns.tolist()
    exog_var_names = [v for v in exog_vars if v in remaining_vars]
    endog_var_names = [v for v in endog_vars if v in remaining_vars]
    
    X = data_clean.values
    exog_indices = [data_clean.columns.tolist().index(v) for v in exog_var_names]
    endog_indices = [data_clean.columns.tolist().index(v) for v in endog_var_names]
    sink_index = data_clean.columns.tolist().index(sink_var)
    
    X_removed_exog, W_endog_dict = remove_effect(X, exog_indices, return_coefs=True)
    del W_endog_dict[sink_index]
    
    W_endog = pd.DataFrame([W_endog_dict[i] for i in endog_indices],
        index=endog_var_names, columns=exog_var_names)
    
    for edge in forbidden_edges:
        if edge['treatment'] in exog_var_names and edge['outcome'] in endog_var_names:
            W_endog.loc[edge['outcome'], edge['treatment']] = 0.0
    
    X_endog = X_removed_exog[:, endog_indices]
    
    no_paths = []
    for edge in forbidden_edges:
        if edge['treatment'] in endog_var_names and edge['outcome'] in endog_var_names:
            from_idx = endog_var_names.index(edge['treatment'])
            to_idx = endog_var_names.index(edge['outcome'])
            no_paths.append([from_idx, to_idx])
    
    if no_paths:
        prior_knowledge = make_prior_knowledge(
            n_variables=len(endog_var_names),
            no_paths=no_paths
        )
        model = lingam.DirectLiNGAM(prior_knowledge=prior_knowledge)
    else:
        model = lingam.DirectLiNGAM()
    
    model.fit(X_endog)
    
    B_endog = pd.DataFrame(model.adjacency_matrix_, 
        index=endog_var_names, columns=endog_var_names)
    
    sink_model = LogisticRegression(max_iter=10000)
    sink_model.fit(X_removed_exog[:, endog_indices], X[:, sink_index])
    
    W_sink = pd.DataFrame(sink_model.coef_, 
        index=[sink_var], columns=endog_var_names)
    
    B_exog = pd.DataFrame(np.eye(len(exog_var_names)) - 1,
        index=exog_var_names, columns=exog_var_names)
    B_exog[B_exog == -1] = np.nan
    
    all_var_names = exog_var_names + endog_var_names + [sink_var]
    adj = pd.DataFrame(0.0, index=all_var_names, columns=all_var_names)
    adj.loc[B_exog.index, B_exog.columns] = B_exog
    adj.loc[W_endog.index, W_endog.columns] = W_endog
    adj.loc[B_endog.index, B_endog.columns] = B_endog
    adj.loc[W_sink.index, W_sink.columns] = W_sink
    
    return adj

def make_labeled_graph(matrix, labels, value_label):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    color = 'red' if value_label == 'Weight' else 'blue'
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            val = matrix.iloc[i, j]
            
            if abs(val) > 0 and not np.isnan(val):
                edge_label = f'<<FONT COLOR="{color}">{val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=edge_label)
    
    return d

def make_dual_value_graph(adj_matrix, effect_matrix, labels):
    import graphviz
    
    d = graphviz.Digraph(engine='dot')
    
    for label in labels:
        d.node(label, label)
    
    for i, outcome in enumerate(labels):
        for j, treatment in enumerate(labels):
            lingam_val = adj_matrix.iloc[i, j]
            effect_val = effect_matrix.iloc[i, j]
            
            if abs(lingam_val) > 0 and not np.isnan(lingam_val):
                label_text = f'<<FONT COLOR="blue"> {lingam_val:.2f}</FONT><BR/><FONT COLOR="red">{effect_val:.2f}</FONT>>'
                d.edge(treatment, outcome, label=label_text)
    
    return d

def analyze_edges_from_matrix(adj_matrix, data):
    edges_df = extract_edges(adj_matrix, min_edge_strength=0.0)
    
    results = []
    
    for idx, edge in edges_df.iterrows():
        treatment = edge['treatment']
        outcome = edge['outcome']
        lingam_weight = edge['lingam_weight']
        
        result = estimate_edge_effect(data, treatment, outcome)
        
        if result:
            result['lingam_weight'] = lingam_weight
            results.append(result)
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    results_df['significant'] = results_df['pvalue'] < 0.05
    
    return results_df

def iterative_pruning_with_lingam(data, exog_vars, endog_vars, sink_var, experiment_name, significance_level=0.05):
    print(f"{'='*80}")
    print(f"ITERATIVE PRUNING WITH LINGAM RERUN: {experiment_name}")
    print(f"{'='*80}")
    
    forbidden_edges = []
    edge_attempt_tracker = {}  # Track {(treatment, outcome): [(pvalue, effect), ...]}
    max_unchanged_attempts = 5  # Stop after 5 attempts with no change
    iteration = 0
    all_iterations = []
    all_adj_matrices = []
    all_effect_matrices = []
    
    while True:
        iteration += 1
        print(f"{'='*80}")
        print(f"ITERATION {iteration}")
        print(f"{'='*80}")
        print(f"Forbidden edges so far: {len(forbidden_edges)}")
        
        print("Running LiNGAM with constraints")
        adj_matrix = run_lingam_with_constraints(data, exog_vars, endog_vars, sink_var, forbidden_edges)
        
        all_adj_matrices.append((iteration, adj_matrix.copy()))
        
        print("Running DoWhy on all edges")
        results_df = analyze_edges_from_matrix(adj_matrix, data)
        
        if results_df is None or len(results_df) == 0:
            print("No edges remaining to analyze")
            break
        
        effect_matrix = adj_matrix.copy()
        for idx, row in results_df.iterrows():
            effect_matrix.loc[row['outcome'], row['treatment']] = row['effect']
        
        all_effect_matrices.append((iteration, effect_matrix.copy()))
        
        num_edges = len(results_df)
        num_significant = results_df['significant'].sum()
        num_nonsignificant = num_edges - num_significant
        
        print(f"Total edges: {num_edges}")
        print(f"Significant edges (p < {significance_level}): {num_significant}")
        print(f"Non-significant edges: {num_nonsignificant}")
        
        results_df['iteration'] = iteration
        all_iterations.append(results_df)
        
        nonsig_edges = results_df[~results_df['significant']]
        
        nonsig_edges_maskable = nonsig_edges[
            ((nonsig_edges['treatment'].isin(endog_vars)) & 
             ((nonsig_edges['outcome'].isin(endog_vars)) | (nonsig_edges['outcome'] == sink_var))) |
            ((nonsig_edges['treatment'].isin(exog_vars)) & (nonsig_edges['outcome'].isin(endog_vars)))
        ]
        
        if len(nonsig_edges_maskable) == 0:
            print(f"All edges are significant! Pruning complete.")
            break
        
        worst_edge = nonsig_edges_maskable.loc[nonsig_edges_maskable['pvalue'].idxmax()]
        
        print(f"Removing edge with highest p-value:")
        print(f"  {worst_edge['treatment']} -> {worst_edge['outcome']}")
        print(f"  p-value: {worst_edge['pvalue']:.6f}")
        print(f"  effect: {worst_edge['effect']:.6f}")
        
        # Check for infinite loop (same edge with unchanging values repeatedly)
        edge_key = (worst_edge['treatment'], worst_edge['outcome'])
        edge_values = (worst_edge['pvalue'], worst_edge['effect'])
        
        # Initialize tracker for this edge if not exists
        if edge_key not in edge_attempt_tracker:
            edge_attempt_tracker[edge_key] = []
        
        # Check if values have changed from last attempt
        if len(edge_attempt_tracker[edge_key]) > 0:
            last_pvalue, last_effect = edge_attempt_tracker[edge_key][-1]
            # Check if values are essentially unchanged (within small tolerance)
            values_unchanged = (abs(worst_edge['pvalue'] - last_pvalue) < 1e-6 and 
                              abs(worst_edge['effect'] - last_effect) < 1e-6)
        else:
            values_unchanged = False
        
        # Add current values to tracker
        edge_attempt_tracker[edge_key].append(edge_values)
        
        # Count consecutive unchanged attempts
        unchanged_count = 0
        if len(edge_attempt_tracker[edge_key]) > 1:
            for i in range(len(edge_attempt_tracker[edge_key]) - 1, 0, -1):
                curr_p, curr_e = edge_attempt_tracker[edge_key][i]
                prev_p, prev_e = edge_attempt_tracker[edge_key][i-1]
                if abs(curr_p - prev_p) < 1e-6 and abs(curr_e - prev_e) < 1e-6:
                    unchanged_count += 1
                else:
                    break
        
        # If edge has been attempted 5 times with no change, skip it
        if unchanged_count >= max_unchanged_attempts - 1:
            print(f"WARNING: Edge {worst_edge['treatment']} -> {worst_edge['outcome']} unchanged after {unchanged_count + 1} attempts.")
            print(f"  p-value: {worst_edge['pvalue']:.6f}, effect: {worst_edge['effect']:.6f}")
            print(f"  Skipping this edge and moving to next worst edge...")
            
            # Mark this specific edge as permanently forbidden (don't try again)
            new_edge = {
                'treatment': worst_edge['treatment'],
                'outcome': worst_edge['outcome']
            }
            
            # Check if already in forbidden list
            if new_edge not in forbidden_edges:
                forbidden_edges.append(new_edge)
            
            # Try to find the next worst non-significant edge to remove
            # Remove the current worst edge from consideration
            nonsig_edges_remaining = nonsig_edges_maskable[
                ~((nonsig_edges_maskable['treatment'] == worst_edge['treatment']) & 
                  (nonsig_edges_maskable['outcome'] == worst_edge['outcome']))
            ]
            
            if len(nonsig_edges_remaining) > 0:
                # Continue with next worst edge in next iteration
                print(f"  Will try next worst edge in next iteration...")
                continue
            else:
                print(f"  No other non-significant edges to try. Stopping pruning.")
                break
        
        # Normal case: add edge to forbidden list
        new_edge = {
            'treatment': worst_edge['treatment'],
            'outcome': worst_edge['outcome']
        }
        
        if new_edge in forbidden_edges:
            print(f"WARNING: Edge {new_edge['treatment']} -> {new_edge['outcome']} already in forbidden list (shouldn't happen).")
            print(f"Stopping pruning.")
            break
            
        forbidden_edges.append(new_edge)
    
    print(f"{'='*80}")
    print(f"PRUNING COMPLETE AFTER {iteration} ITERATIONS")
    print(f"{'='*80}")
    
    final_results = all_iterations[-1] if all_iterations else None
    
    if final_results is not None:
        print(f"Final network has {len(final_results)} edges")
        print(f"All edges are significant (p < {significance_level})")
        
        combined_results = pd.concat(all_iterations, ignore_index=True)
        combined_results.to_csv(f'{experiment_name}_results.csv', index=False)
        print(f"Saved: {experiment_name}_results.csv")
        
        all_adj_dfs = []
        for iter_num, adj_mat in all_adj_matrices:
            adj_stacked = adj_mat.stack().reset_index()
            adj_stacked.columns = ['outcome', 'treatment', 'weight']
            adj_stacked['iteration'] = iter_num
            all_adj_dfs.append(adj_stacked)
        
        combined_adjacency = pd.concat(all_adj_dfs, ignore_index=True)
        combined_adjacency.to_csv(f'{experiment_name}_adjacency.csv', index=False)
        print(f"Saved: {experiment_name}_adjacency.csv")
        
        print("Generating PDFs")
        
        try:
            from PyPDF2 import PdfMerger
            import os
            
            merger_lingam = PdfMerger()
            merger_dowhy = PdfMerger()
            temp_files = []
            
            for iter_num, adj_mat in all_adj_matrices:
                dot = make_labeled_graph(adj_mat, adj_mat.columns.tolist(), 'Weight')
                temp_file = f'{experiment_name}_temp_lingam_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_lingam.append(f'{temp_file}.pdf')
            
            for iter_num, eff_mat in all_effect_matrices:
                dot = make_labeled_graph(eff_mat, eff_mat.columns.tolist(), 'Effect')
                temp_file = f'{experiment_name}_temp_dowhy_iteration_{iter_num}'
                dot.render(temp_file, format='pdf', cleanup=True)
                temp_files.append(f'{temp_file}.pdf')
                merger_dowhy.append(f'{temp_file}.pdf')
            
            output_pdf_lingam = f'{experiment_name}_iterations_lingam.pdf'
            merger_lingam.write(output_pdf_lingam)
            merger_lingam.close()
            print(f"Saved: {output_pdf_lingam}")
            
            output_pdf_dowhy = f'{experiment_name}_iterations_dowhy.pdf'
            merger_dowhy.write(output_pdf_dowhy)
            merger_dowhy.close()
            print(f"Saved: {output_pdf_dowhy}")
            
            for temp_file in temp_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            

            # Create first iteration graphs (initial state before pruning)
            print("Creating first iteration graphs")
            first_adj_lingam = all_adj_matrices[0][1].copy()
            first_adj_dowhy = all_effect_matrices[0][1].copy()
            
            dot_first_lingam = make_labeled_graph(first_adj_lingam, first_adj_lingam.columns.tolist(), 'Weight')
            dot_first_lingam.render(f'{experiment_name}_first_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_lingam.pdf")
            
            dot_first_dowhy = make_labeled_graph(first_adj_dowhy, first_adj_dowhy.columns.tolist(), 'Effect')
            dot_first_dowhy.render(f'{experiment_name}_first_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dowhy.pdf")
            
            dot_dual_first = make_dual_value_graph(first_adj_lingam, first_adj_dowhy, first_adj_lingam.columns.tolist())
            dot_dual_first.render(f'{experiment_name}_first_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_first_dual.pdf")
            
            # Save first iteration results to CSV
            if len(all_iterations) > 0:
                first_results = all_iterations[0].copy()
                first_results.to_csv(f'{experiment_name}_first_results.csv', index=False)
                print(f"Saved: {experiment_name}_first_results.csv")
            
            print("Creating final graphs")
            significant_edges = final_results[final_results['significant']]
            
            sig_adj_lingam = all_adj_matrices[-1][1].copy()
            sig_adj_dowhy = all_effect_matrices[-1][1].copy()
            
            for idx, row in final_results[~final_results['significant']].iterrows():
                sig_adj_lingam.loc[row['outcome'], row['treatment']] = 0.0
                sig_adj_dowhy.loc[row['outcome'], row['treatment']] = 0.0
            
            dot_final_lingam = make_labeled_graph(sig_adj_lingam, sig_adj_lingam.columns.tolist(), 'Weight')
            dot_final_lingam.render(f'{experiment_name}_final_lingam', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_lingam.pdf")
            
            dot_final_dowhy = make_labeled_graph(sig_adj_dowhy, sig_adj_dowhy.columns.tolist(), 'Effect')
            dot_final_dowhy.render(f'{experiment_name}_final_dowhy', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dowhy.pdf")
            
            dot_dual_final = make_dual_value_graph(sig_adj_lingam, sig_adj_dowhy, sig_adj_lingam.columns.tolist())
            dot_dual_final.render(f'{experiment_name}_final_dual', format='pdf', cleanup=True)
            print(f"Saved: {experiment_name}_final_dual.pdf")
            
            print(f"Final network has {len(significant_edges)} significant edges")
            
        except ImportError:
            print("PyPDF2 not installed. Install with 'pip install PyPDF2' to create combined PDF.")
    
    return adj_matrix, final_results, forbidden_edges

data_no_cirrhosis = pd.read_csv('data_no_cirrhosis.csv')
data_with_cirrhosis = pd.read_csv('data_with_cirrhosis.csv')

organ_injury_mediators = ['cardio28', 'cns28', 'coag28', 'hepatic28', 'icufd',
                          'orgfree28', 'pulmon28', 'renal28', 'vfd']

additional_mediators = ['album_12', 'bili7', 'cumfl_8', 'fio2abg_7', 'flbal_7',
    'fluidin_8', 'gcs_7', 'gluc_2', 'goalrate_0', 'hgb_7', 'mg_2', 'minvent_4', 
    'motorc_14', 'oi_3', 'peep_1', 'pf7', 'phabg_4', 'phos_1', 'pip_7', 'potas_2', 'tidal_12']

all_mediators = organ_injury_mediators + additional_mediators
sink_var_name = 'death90'

print("" + "="*80)
print("RUNNING CIRRHOSIS-STRATIFIED ANALYSIS FOR ALPHA = 0.05")
print("="*80)

print("" + "="*80)
print("NO CIRRHOSIS GROUP")
print("="*80)

data_no_cirrhosis_continuous = data_no_cirrhosis[data_no_cirrhosis['drinks_per_week'].notna()].copy()
data_no_cirrhosis_continuous = data_no_cirrhosis_continuous[['drinks_per_week'] + all_mediators + [sink_var_name]].dropna(subset=['drinks_per_week'])

final_adj_no_cirrhosis, results_no_cirrhosis, forbidden_no_cirrhosis = iterative_pruning_with_lingam(
    data=data_no_cirrhosis_continuous,
    exog_vars=['drinks_per_week'],
    endog_vars=all_mediators,
    sink_var=sink_var_name,
    experiment_name='cirrhosis_absent_continuous_drinks_per_week',
    significance_level=0.05
)

# CATEGORICAL GROUPS COMMENTED OUT - Using continuous drinks_per_week only
# data_no_cirr_cat = data_no_cirrhosis[data_no_cirrhosis['alcohol_category'].notna()].copy()

# data_no_cirr_group1 = data_no_cirr_cat.copy()
# data_no_cirr_group1['alcohol_binary'] = data_no_cirr_group1['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Heavy Drinker': 1
# })
# data_no_cirr_group1 = data_no_cirr_group1.dropna(subset=['alcohol_binary'])
# data_no_cirr_group1['alcohol_binary'] = data_no_cirr_group1['alcohol_binary'].astype(int)

# final_adj_no_cirr_g1, results_no_cirr_g1, forbidden_no_cirr_g1 = iterative_pruning_with_lingam(
#     data=data_no_cirr_group1,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_absent_group1_nondrinker_vs_heavy',
#     significance_level=0.05
# )

# data_no_cirr_group3 = data_no_cirr_cat.copy()
# data_no_cirr_group3['alcohol_binary'] = data_no_cirr_group3['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1,
#     'Heavy Drinker': 1
# })
# data_no_cirr_group3 = data_no_cirr_group3.dropna(subset=['alcohol_binary'])
# data_no_cirr_group3['alcohol_binary'] = data_no_cirr_group3['alcohol_binary'].astype(int)

# final_adj_no_cirr_g3, results_no_cirr_g3, forbidden_no_cirr_g3 = iterative_pruning_with_lingam(
#     data=data_no_cirr_group3,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_absent_group3_nondrinker_vs_anydrinker',
#     significance_level=0.05
# )

# data_no_cirr_group4 = data_no_cirr_cat.copy()
# data_no_cirr_group4['alcohol_binary'] = data_no_cirr_group4['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1
# })
# data_no_cirr_group4 = data_no_cirr_group4.dropna(subset=['alcohol_binary'])
# data_no_cirr_group4['alcohol_binary'] = data_no_cirr_group4['alcohol_binary'].astype(int)

# final_adj_no_cirr_g4, results_no_cirr_g4, forbidden_no_cirr_g4 = iterative_pruning_with_lingam(
#     data=data_no_cirr_group4,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_absent_group4_nondrinker_vs_moderate',
#     significance_level=0.05
# )

print("" + "="*80)
print("WITH CIRRHOSIS GROUP")
print("="*80)

data_with_cirrhosis_continuous = data_with_cirrhosis[data_with_cirrhosis['drinks_per_week'].notna()].copy()
data_with_cirrhosis_continuous = data_with_cirrhosis_continuous[['drinks_per_week'] + all_mediators + [sink_var_name]].dropna(subset=['drinks_per_week'])

final_adj_with_cirrhosis, results_with_cirrhosis, forbidden_with_cirrhosis = iterative_pruning_with_lingam(
    data=data_with_cirrhosis_continuous,
    exog_vars=['drinks_per_week'],
    endog_vars=all_mediators,
    sink_var=sink_var_name,
    experiment_name='cirrhosis_present_continuous_drinks_per_week',
    significance_level=0.05
)

# CATEGORICAL GROUPS COMMENTED OUT - Using continuous drinks_per_week only
# data_with_cirr_cat = data_with_cirrhosis[data_with_cirrhosis['alcohol_category'].notna()].copy()

# data_with_cirr_group1 = data_with_cirr_cat.copy()
# data_with_cirr_group1['alcohol_binary'] = data_with_cirr_group1['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Heavy Drinker': 1
# })
# data_with_cirr_group1 = data_with_cirr_group1.dropna(subset=['alcohol_binary'])
# data_with_cirr_group1['alcohol_binary'] = data_with_cirr_group1['alcohol_binary'].astype(int)

# final_adj_with_cirr_g1, results_with_cirr_g1, forbidden_with_cirr_g1 = iterative_pruning_with_lingam(
#     data=data_with_cirr_group1,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_present_group1_nondrinker_vs_heavy',
#     significance_level=0.05
# )

# data_with_cirr_group3 = data_with_cirr_cat.copy()
# data_with_cirr_group3['alcohol_binary'] = data_with_cirr_group3['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1,
#     'Heavy Drinker': 1
# })
# data_with_cirr_group3 = data_with_cirr_group3.dropna(subset=['alcohol_binary'])
# data_with_cirr_group3['alcohol_binary'] = data_with_cirr_group3['alcohol_binary'].astype(int)

# final_adj_with_cirr_g3, results_with_cirr_g3, forbidden_with_cirr_g3 = iterative_pruning_with_lingam(
#     data=data_with_cirr_group3,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_present_group3_nondrinker_vs_anydrinker',
#     significance_level=0.05
# )

# data_with_cirr_group4 = data_with_cirr_cat.copy()
# data_with_cirr_group4['alcohol_binary'] = data_with_cirr_group4['alcohol_category'].map({
#     'Non-Drinker': 0,
#     'Moderate Drinker': 1
# })
# data_with_cirr_group4 = data_with_cirr_group4.dropna(subset=['alcohol_binary'])
# data_with_cirr_group4['alcohol_binary'] = data_with_cirr_group4['alcohol_binary'].astype(int)

# final_adj_with_cirr_g4, results_with_cirr_g4, forbidden_with_cirr_g4 = iterative_pruning_with_lingam(
#     data=data_with_cirr_group4,
#     exog_vars=['alcohol_binary'],
#     endog_vars=all_mediators,
#     sink_var=sink_var_name,
#     experiment_name='cirrhosis_present_group4_nondrinker_vs_moderate',
#     significance_level=0.05
# )

print("" + "="*80)
print("CIRRHOSIS-STRATIFIED ANALYSIS COMPLETE")
print("="*80)

/home/arslanbisharat/anaconda3/envs/arslan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RUNNING CIRRHOSIS-STRATIFIED ANALYSIS FOR ALPHA = 0.05
NO CIRRHOSIS GROUP
ITERATIVE PRUNING WITH LINGAM RERUN: cirrhosis_absent_continuous_drinks_per_week
ITERATION 1
Forbidden edges so far: 0
Running LiNGAM with constraints
Running DoWhy on all edges
Total edges: 42
Significant edges (p < 0.05): 33
Non-significant edges: 9
Removing edge with highest p-value:
  drinks_per_week -> hepatic28
  p-value: 0.928626
  effect: 0.001977
ITERATION 2
Forbidden edges so far: 1
Running LiNGAM with constraints
Running DoWhy on all edges
Total edges: 41
Significant edges (p < 0.05): 33
Non-significant edges: 8
Removing edge with highest p-value:
  drinks_per_week -> cns28
  p-value: 0.906284
  effect: 0.003085
ITERATION 3
Forbidden edges so far: 2
Running LiNGAM with constraints
Running DoWhy on all edges
Total edges: 40
Significant edges (p < 0.05): 33
Non-significant edges: 7
Removing edge with highest p-value:
  drinks_per_week -> icufd
  p-value: 0.851584
  effect: -0.004295
ITERATION 4
Forbidden